In [1]:
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from umap import UMAP
from hdbscan import HDBSCAN

d:\softwares\python\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\softwares\python\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\softwares\python\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument was

# 加载数据

In [2]:
# step1 加载文件
with open('../../data/切词.txt', 'r', encoding='utf-8') as file:
  docs = file.readlines()
print('条数: ', len(docs))
print('预览第一条: ', docs[0])

vectorizer_model = None

条数:  1000
预览第一条:  文旅文 创看 洛阳 河南省 文旅文创 发展 大会 本次 大会 安排 项目 签约 主要 方面 内容 一是 文旅 产业 项目 签约 截至 目前 梳理 重点 文旅 项目 投资总额 525.6 亿元 遴选 重大项目 进行 现场 签约 投资总额 365.8 亿元 项目 包括 文物 数字化 开发 文化 创意 园区 建设 文化 项目 涵盖 旅游 度假区 建设 旅游 酒店 民宿 打造 旅游 项目 既有 旅游 景区 开发 商旅 综合体 建设 传统 业态 项目 宇宙 基地 沉浸 演艺 业态 项目 充分体现 我省 文化 旅游 发展 特点 趋势 二是 引客 入豫 项目 签约 主要 我省 文旅 部门 文旅 企业 头部 旅行 知名 OTA 平台 重点 客源地 文旅 部门 签订 引客 入豫 协议 持续 拓展 省外 客源 市场



# 创建

In [3]:
# 1. 词向量模型，同时加载本地训练好的词向量
embedding_model = pipeline("feature-extraction", model="bert-base-chinese") # 使用bert-base-chinese

# 2. 创建UMAP降维模型
umap_model = UMAP(
  n_neighbors=15,
  n_components=5,
  min_dist=0.0,
  metric='cosine',
  random_state=42  # ⚠️ 防止随机 https://maartengr.github.io/BERTopic/faq.html
)

# 3. 创建聚类模型
hdbscan_model = HDBSCAN(min_cluster_size=40)


# 4. 创建分词模型
vectorizer_model = CountVectorizer() # 因为我们已经分好词了，所以这里不需要传入分词函数了

embeddings = np.load('../../data/embedding_bbc.npy') # 使用bert-base-chinese向量
print(embeddings.shape)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(1000, 768)


In [4]:
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  hdbscan_model=hdbscan_model,
  umap_model=umap_model,
)

topics = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,501,-1_洛阳_旅游_景区_文化,"[洛阳, 旅游, 景区, 文化, 河南, 游客, 城市, 洛阳市, 中国, 历史]",[河南 多家 景区 陆续 开园 台风 杜苏芮 强度 逐渐 减弱 河南 景区 陆续 发布 开园...
1,0,216,0_洛阳_中国_石窟_龙门石窟,"[洛阳, 中国, 石窟, 龙门石窟, 旅游, 世界, 艺术, 旅行, 景区, 位于]",[龙门石窟 中国 石刻 艺术 宝库 现为 世界 文化遗产 全国 重点 文物保护 单位 国家 ...
2,1,100,1_洛阳_活动_景区_免费,"[洛阳, 活动, 景区, 免费, 旅游, 高速, 门票, 时间, 白云山, 文化]",[庆祝 河南省 老君山 文化 旅游 集团 重金 打造 老君山 景区 周年 河南 栾川 老君山...
3,2,99,2_旅游_文化_洛阳_发展,"[旅游, 文化, 洛阳, 发展, 城市, 项目, 河南, 文旅, 活动, 中国]",[今日 文旅 行业 信息 洛阳 方特 项目 签约 投资 亿元 洛阳 发布 公众 消息 近日 ...
4,3,42,3_洛阳_旅游_地方_很多,"[洛阳, 旅游, 地方, 很多, 一天, 时间, 已经, 一点, 古城, 酒店]",[记录 一下 6.7 洛阳 这次 洛阳 算是 放空 洛阳 一座 富裕 小城 三天 两夜 很多...
5,4,42,4_旅客_洛阳_郑州_高速,"[旅客, 洛阳, 郑州, 高速, 景区, 客流, 旅游, 出行, 发送, 通行]",[今天 郑州 铁路 预计 发送 旅客 51.7 万人 管内 客流量 激增 中国 铁路 郑州 ...


In [5]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)